<img src="./img/HWNI_logo.svg"/>

# Organizing Data with Pandas

In neuroscience, we often work with complicated datasets. For example, a full "data point" from a neuroscience experiment might include several numbers (input stimulus and neural response) along with a host of metadata -- subject ID, brain region, genotype, experiment date, and so on. Trying to manage this with a collection of arrays is an exercise in frustration, and dictionaries hardly improve the situtation.

Instead, we can borrow a tool from other data-heavy sciences: the _data frame_. A data frame is like a matrix, in that it contains information in rows and columns, but it is even more like a table, in that the rows and columns have names. We'll dive deeper into this below.

The premier package implementing data frames in Python is
[pandas](http://pandas.pydata.org/pandas-docs/stable/overview.html).
If you've worked with tables in another context, e.g. in the R language,
or if you're very comfortable with Python,
you can check out the
[10 minutes to pandas](http://pandas.pydata.org/pandas-docs/stable/10min.html)
mini-tutorial on the pandas website, which covers lots of material at a brisk pace.

The following tutorial is indebted to
[Brandon Rhodes' PyCon tutorial](https://www.youtube.com/watch?v=5JnMutdy6Fw).
That tutorial takes between four and six hours,
but it does an excellent job in building a foundation for working with data frames,
and Mr. Rhodes is a fabulous teacher,
so it's very worth your while if you have the time.

Let's start by importing pandas.

In [81]:
# pandas - "DataFrames" to organize our data
import pandas as pd

The next cell changes the formatting of the data frames when they're printed to the screen by Jupyter. Comment out these lines and then re-run the notebook to see the default style.

In [3]:
# this makes our tables easier to read
from IPython.core.display import HTML
css = open('./css/style-table.css').read()
#this line has to be the last in its cell
HTML('<style>{}</style>'.format(css)) 

We'll be using a dataset from [IMDb](http://www.imdb.com/), the Internet Movie Database, `titles.csv`, which contains the title and year of release for almost all movies that had a commerical release. Like all of the data we'll be working with in this course, the data is in the _comma-separated values_ or `.csv` format. That means that the data is stored as a text file, just like the humble `.txt` format, but with the following structure:

```
column1Name,column2Name
row1column1,row1column2
row2column1,row2column2
```

and so on.


We load `titles` in below, then use the `.head` method to look at the first few entries.

In [36]:
titles = pd.DataFrame.from_csv('data/titles.csv',
                               index_col = None,
                               encoding='utf-8')

head = titles.head()
head

,title,year
0,Lemmy pour les dames,1962
1,Desperate Moves,1981
2,Malice: Emergence,2016
3,Raven's Ridge,1997
4,Palan ibyeolui geulshi,1968


To look at a random collection of entries, we can use the `.sample` method:

In [34]:
sample = titles.sample(10)

sample

,title,year
220908,Superseven Is Still My Name,2015
122219,Goa,2010
53507,Endrendrum Punnagai,2013
8419,Konn,2014
205340,Dream Cash,2012
121844,22 Hands,2006
159127,Jucy,2010
76625,Sbank,2013
68730,Bahu Beta Aur Maa,1992
145103,Chinaman Wu-Ping: Stranger Out of Canton,2016


Note that `sample` and `head` were also data frames.

## Indexing into Data Frames

We can access one column at a time from a data frame in one of two ways. If the name of a column has no spaces, then we can type `dataframe.columnName`. When that doesn't work, for example if the column name has spaces in it, we can use the same syntax we use to access dictionaries: `dataframe["column name with spaces"]`. Note that in the second case, we had to provide the column name as a string.

We usually use the `.columnName` syntax when we're directly accessing columns by hand and the `["columnName"]` syntax when we're writing code that accesses columns for us, as below. Can you see why the third example couldn't be rewritten as `sample.column`?

In [82]:
print(sample.title)

# we'll put a separator in between each series so they're easier to read.
#    make a string that is =*= repeated 17 times, then print it
print("=*="*17) 

print(sample["title"])

print("=*="*17)

column = "title"
print(sample[column])

220908                 Superseven Is Still My Name
122219                                         Goa
53507                          Endrendrum Punnagai
8419                                          Konn
205340                                  Dream Cash
121844                                    22 Hands
159127                                        Jucy
76625                                        Sbank
68730                            Bahu Beta Aur Maa
145103    Chinaman Wu-Ping: Stranger Out of Canton
Name: title, dtype: object
=*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*=
220908                 Superseven Is Still My Name
122219                                         Goa
53507                          Endrendrum Punnagai
8419                                          Konn
205340                                  Dream Cash
121844                                    22 Hands
159127                                        Jucy
76625                                        Sbank
687

The resulting object is called a `Series`. Once we have a `Series`, we can manipulate the data inside very easily. Mathematical operations like adding and logical operations like checking for equality get applied to each element of the `Series`, which gives us a new `Series`. For example, we can floor divide by 10 (divide and throw away the remainder, symbolized by `//`) and then multiply by 10 to figure out the decades of the movies in our sample, and then we can check whether the decade is equal to `2000` to see if any naughty-aughties movies showed up.

In [75]:
decades = sample.year // 10 * 10

print(decades)
print("=*="*8)

isDoubleZeros = (decades == 2000)
print(isDoubleZeros)

220908    2010
122219    2010
53507     2010
8419      2010
205340    2010
121844    2000
159127    2010
76625     2010
68730     1990
145103    2010
Name: year, dtype: int64
=*==*==*==*==*==*==*==*=
220908    False
122219    False
53507     False
8419      False
205340    False
121844     True
159127    False
76625     False
68730     False
145103    False
Name: year, dtype: bool


Notice the `dtype` on extra line printed after each `Series`. `dtype` is short for "**d**ata **type**". For the decades, the type is `int64`, or "64-bit integer". For the series telling us whether each datapoint comes from the 2000s or not, the type is `bool`, or "boolean".

`Series` of booleans are very important for working with data frames. A boolean is a logical value -- it can be either `True` or `False`, not anything else. They are named after [George Boole](https://en.wikipedia.org/wiki/George_Boole), who wrote down the rules for manipulating logical values that underlie digital logic and thus modern computation. Booleans are also sometimes represented as 0s and 1s -- 0 being `False`, 1 being `True`.

`Series` of booleans are important because they can be used to index into, that is, ask for elements from, a data frame. If we use a `Series` of booleans to index into a data frame, we will get out the rows from that data frame corresponding to `True`s in the `Series`. 

We can use `Series` of booleans to pull out subsets of our data -- for example, perhaps we want to know which movies in our sample came from the 2000s. First, we build a series, like `isDoubleZeros` above, that corresponds to testing whether a row falls into our subset. Then, we use that series to index into the data frame. We index into data frames the same way we index into lists or dictionaries: using square brackets.

In [61]:
sample[isDoubleZeros]

,title,year
121844,22 Hands,2006


We don't have to define the `Series` as a variable. Instead, we can put the logical statement inside the brackets, where we would've put the variable name, and the output `Series` will get used for indexing.

Below, we pull out all of the movies from the main `titles` data frame that were made in the 19th century.

In [62]:
titles[titles.year < 1900]

,title,year
65423,The Startled Lover,1898
118916,Reproduction of the Corbett and Jeffries Fight,1899
165917,Miss Jerry,1894


We can also check logical statements on strings. The syntax below will let you pull out your [favorite movie](https://en.wikipedia.org/wiki/Rosencrantz_%26_Guildenstern_Are_Dead_%28film%29) from the database. Watch out, though: you need to get the title exactly right (spelling, punctuation, and capitals) or else you'll find nothing.

In [78]:
favoriteMovieTitle = "Rosencrantz & Guildenstern Are Dead" #put your favorite movie's title here

titles[titles.title == favoriteMovieTitle]

,title,year
52524,Rosencrantz & Guildenstern Are Dead,1990


Python comes with a bunch of [string methods](), like `contains` and `lower`, that are useful in this situation. To use them, append `.str` to the name of the column and then `.methodName` to use the method you're interested in. 

For example, we can find all movies containing the word *rosencrantz*, regardless of capitalization, using the cell below. If you had trouble finding your favorite movie above, try adapting the cell below. You might also be interested in the string method `startswith`.

In [72]:
titles[titles.title.str.lower().str.contains("rosencrantz")]

,title,year
52524,Rosencrantz & Guildenstern Are Dead,1990
175518,Rosencrantz and Guildenstern Are Undead,2009


## Other Useful Skills

This is, of course, just the tiniest bit of what is possible with pandas. 

As we go through the course, we'll introduce additional pieces as necessary. If you can't wait to get started, you might be interested in [`groupby`](http://pandas.pydata.org/pandas-docs/stable/groupby.html), which lets you break a column into subsets based on the values in another column ([example here](http://chrisalbon.com/python/pandas_apply_operations_to_groups.html)).

Even everything we learn in this course will only scratch the surface, and it's likely your data will need some of pandas' advanced capabilities. You can start with
[Brandon Rhodes' PyCon tutorial](https://www.youtube.com/watch?v=5JnMutdy6Fw),
which provides a solid foundation for learning from the pandas documentation or from
intermediate and advanced tutorials and blog posts.

The Jupyter notebook [Plotting with Seaborn](./04 - Plotting with Seaborn.ipynb) in this folder covers the `seaborn` plotting package, which uses pandas data frames to simplify statistical plotting. We'll be using seaborn a lot, so make sure to check it out!